In [1]:
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import pandas as pd

%matplotlib inline

In [2]:
import os
import glob
import pandas as pd
os.chdir("./data")

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('./2-processing/bnci-001-2014*.{}'.format(extension))]

In [4]:
#combine all files in the list
x = pd.concat([pd.read_csv(f) for f in all_filenames ])

Y = pd.read_csv('labels_004_1.csv', header=None)

In [8]:
x.shape

(4149, 23)

In [6]:
Y.shape

(1, 20737)

In [ ]:
np.count_nonzero(Y)

In [ ]:
labels = Y[:461]

In [ ]:
labels.shape

In [ ]:
labels = labels.to_numpy()[0]

In [ ]:
labels = labels[:461]

In [ ]:
labels.shape

In [ ]:
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, labels, test_size=0.1, random_state=seed)

# Test options and evaluation metric

scoring = 'accuracy'

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB(var_smoothing=0.001)))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)



In [ ]:
nb = models[4][1]
nb.fit(X_train,labels[:414])

In [ ]:
from joblib import dump, load
dump(nb, './models/modelNB.joblib') 

In [ ]:
clf = load('./models/modelNB.joblib') 

In [ ]:
clf.predict(X_train)

In [ ]:
labels.info

In [ ]:
labels.shape

In [ ]:
np.count_nonzero(labels)

In [ ]:
from sklearn.metrics import confusion_matrix